# Ad Optimization Hackathon - Aimpoint Digital

## Hackathon Prompt

For this Hackathon, you will be querying an API that contains data on 5 different ads (ad_1, ad_2, ad_3, ad_4 and ad_5). Each query of the API for a certain ad outputs a '1' or '0'. '1' means the ad was clicked on and '0' means the ad was not clicked on. 


The following code queries the API to output 10 datapoints for all 5 ads in Python. You may also use the following code as starter code.

In [2]:
import requests
import json
import time
N = 10 # Set how many data points you would like. Querying the API for a single data point takes ~ 0.2 s. 
ad_list = ['ad_1', 'ad_2', 'ad_3', 'ad_4', 'ad_5'] 
for ad in ad_list:
    ad_outcome = []
    for i in range (N):
        ad_gen = "https://755vgi76zg.execute-api.us-east-1.amazonaws.com/prod"
        data = {
            "ad": ad
        }

        response = requests.post(ad_gen, json.dumps(data))
        resp = json.loads(response.content)
        generate_ad = resp["body"]
        ad_outcome.append(generate_ad[1])
    print ('ad_outcomes for ', ad, ' = ', ad_outcome)

ad_outcomes for  ad_1  =  ['1', '0', '0', '1', '0', '0', '1', '0', '1', '1']
ad_outcomes for  ad_2  =  ['1', '1', '0', '1', '1', '0', '0', '1', '0', '1']
ad_outcomes for  ad_3  =  ['1', '0', '0', '0', '1', '1', '1', '1', '1', '1']
ad_outcomes for  ad_4  =  ['0', '0', '0', '0', '0', '1', '0', '0', '0', '0']
ad_outcomes for  ad_5  =  ['0', '1', '0', '1', '0', '0', '0', '1', '0', '1']


The following are some generic API instructions if you choose to use a language other than Python.

**URL** : https://755vgi76zg.execute-api.us-east-1.amazonaws.com/prod

**HTTP Method** : POST

**Request Body** : {'ad': 'ad_1'} 

The key ad is the particular ad you want the outcome for. The values can be 'ad_1', 'ad_2', ..., 'ad_5' depending on which ad you're querying.

**Format** : 
The POST method will require the user to send the URL and the Request Body to the API. 

**Response**: '{"statusCode": 200, "body": "[1]"}'

### Considerations

Here are some comments on the distributions and the rewards:

1. The distributions for all 5 ads remain constant and do not change over time.
2. Every time an ad is clicked on, you end up with a reward of +1. We do not have data on whether the ad ends up in conversion.
3. Contestants can make as many API calls as they like. 


### Objective


1. You will submit your well-documented code in an easily executable format.
2. You will also submit a short video (screen-recorded) of your approach and results by the end of the hackathon.
3. Evalution will be done on **'reward obtained per 1000 steps'**. Please report this in the demo and in your code submission.

Good Luck!

## Supplementary Reading (Optional)

### This is some additional information that you might find useful. You are not required to use this for the hackathon and this is optional reading!

For this hackathon, we will be exploring a technique called Multi-Armed Bandits (MABs). 

To understand MABs, let's consider an example. We are all going on a vacation to Las Vegas, and we plan on trying our luck at the casinos. We want to play the slot machines, and as with any form of gambling, we want to maximize our rewards at the end of the day. The casino we go to has $K$ slot machines corresponding to K levers that we can pull. The reward for each machine is drawn from the following reward distributions corresponding to each lever $$B = \{R_1, R_2, R_3, ... ,R_K\} \,\,\,\,\,\, where\, K\, \in\,\, \mathbf{N}^+$$
    
    

Let $\mu_1, ..., \mu_K$ be the true means associated with each of the reward distributions. Obviously, we don't know these values for gambling would then be child's play. We do, however, want to correctly approximate the means of these reward distributions so that we may exploit those levers that correspond to the maximum reward. Ideally, we'd want to do this in as few steps as possible. Formally, the number of steps we have remaining is called *Horizon* ($H$).

How do we quantify how good our strategy is? The quality of our strategy is measured by a metric called *Regret* $(\rho)$ as defined by the following formula $$ \rho = T\mu^* - \sum_{t=1}^{T} \hat{r_t}$$

where $\mu^* = \displaystyle\max_{k} \{\mu_k\}$ or the maximal reward mean across levers and $T$ is the number of pulls on the lever. This is synonymous with the lever offering the greatest reward. We don't know which lever does this presently, but we want to correctly converge towards it. $\hat{r_t}$ is the reward in round $t$. This refers to the reward from a potentially sub-optimal lever we are operating in the present. In plain English, $\rho$ tells us how close/far we are from identifying the optimal lever. If $\rho$ is large, then we are not close to identifying the optimal lever. If $\rho$ is consistently large over time, it means that our algorithm has converged on a sub-optimal solution, and we are stuck there. Ideally, we want $\rho$ to decrease and tend towards 0 over time. 

I'm sure all of you must've realized by now that bandits are an exploration Vs exploitation problem. Explore forever, and you might someday find the optimal lever - but your losses will be too large to recuperate from. Exploit from the start, and you'll easily get stuck at a sub-optimal solution. Winning this hackathon will require dealing with the exploration Vs exploitation dilemma tactfully and this will allow you to minimize cumulative $\rho$.

### Multi-Armed Bandits : A Simple Example

Let's understand how to approach a MAB with an example using sample data. The example and the data for this example has been sourced from [here](https://www.analyticsvidhya.com/blog/2018/09/reinforcement-multi-armed-bandit-scratch-python/). We have 10 different ads that can be run on our website and we some historic information on which ads were clicked by users. '1' represents a click and '0' represents no click. We want to identify which ads have the highest likelihood to be clicked by a new user to our website. This is very similar to the slot machine problem, except we are trying to optimize ads to maximize Click Through Rate (CTR). You can find a sample of the dataset below. The complete dataset can be found [here](https://drive.google.com/file/d/1whkIInL4FKeHg2IfdcbT1j18L26fg9aF/view). Make sure to download the whole dataset in order to implement the code snippets in this notebook.

<p align="center">
  <img width="460" height="460" src="https://cdn.analyticsvidhya.com/wp-content/uploads/2018/09/im_24.jpg">
</p>

### Random Sampling - Simple Approach to MABs

We will start with the simplest solution to the exploration vs exploitation dilemma - random sampling. It can be argued that this isn't a solution at all - but its good place to start from and set a solid baseline to compare other techniques with. Every time a user comes to our website, we randomly show them one of the ads. We do this experiment 10,000 times and record the total reward from this experiment. The following code snippet implements this for you.

In [1]:
# Random Selection

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Ads_Optimisation.csv')

# Implementing Random Selection
import random
random.seed(13)
N = 10000
d = 10
ads_selected = []
total_reward = 0
for n in range(0, N):
    ad = random.randrange(d)
    ads_selected.append(ad)
    reward = dataset.values[n, ad]
    total_reward = total_reward + reward
    
print ('Total Reward from Random Sampling: ', total_reward)

Total Reward from Random Sampling:  1248


We have received a total reward of 1248. Is this any good? We don't know for sure yet. But its very likely a poor, sub-optimal result. We can tackle the exploration vs exploitation dilemma in a manner that is much better than simply picking random actions.